Importing libs

In [ ]:
%matplotlib notebook

import numpy as np
import pandas as pd
from numba import jit

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern, DotProduct

from scipy.integrate import solve_ivp
from scipy import optimize

import arviz as az
from arviz.utils import Numba

import pymc3 as pm
import theano
import theano.tensor as t
from scipy.stats import gaussian_kde
from tqdm import tqdm, trange

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

OUTPUT_PATH = "./Results"

from data_loading import LoadData
from proj_consts import ProjectConsts

Setting seed

In [ ]:
seed = 12345

Plotting functions

In [ ]:
def plotDataset(xdata, ydata, xlabel, ylabel, figname, data_type):
    f = plt.figure(figsize=(9, 7))

    plt.plot(
        xdata,
        ydata,
        marker="o",
        linestyle="",
    )

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid()
    plt.tight_layout()
    plt.show()
    # plt.savefig(f"{OUTPUT_PATH}/" + figname + "_" + str(data_type) + ".pdf")

    
def plotDatasetsComparison(xdata, ydata1, ydata2, leg1, leg2, xlabel, ylabel, figname):
    fig, ax = plt.subplots(figsize=(9, 7))

    plt.plot(
        xdata,
        ydata1,
        label=leg1,
        marker="o",
        markersize=0,
        linestyle="-",
        linewidth=1.0,
    )

    plt.plot(
        xdata,
        ydata2,
        label=leg2,
        marker="o",
        markersize=0,
        linestyle="-",
        linewidth=0.5,
    )

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid()

    plt.tight_layout()
    plt.show()
    # plt.savefig(f"{OUTPUT_PATH}/" + figname + ".pdf")

Retrieving data

In [ ]:
min_confirmed = 5
num_days_data = 210

df_brazil_state_cases = LoadData.getBrazilDataFrame(min_confirmed)
rj_state_cases = LoadData.getBrazilStateDataFrame(df_brazil_state_cases, "RJ")
rj_state_cases = rj_state_cases.head(num_days_data)
print(rj_state_cases)

target_population = ProjectConsts.RJ_STATE_POPULATION

Data regularization

In [ ]:
data_time = rj_state_cases.day.values.astype(np.float64)

dead_individuals = rj_state_cases.dead.values
infected_individuals = rj_state_cases.infected.values
confirmed_cases = rj_state_cases.confirmed.values
recovered_cases = rj_state_cases.recovered.values

original_data = [dead_individuals, infected_individuals, confirmed_cases, recovered_cases]
original_data_reg = np.zeros((4, len(data_time)))

for i, data in enumerate(original_data):
    scale_factor = data[-1] / 100
    data = data / scale_factor

    # Defining the kernel
    if i == 1: # When dealing with new confirmed cases dataset
        kernel = Matern()
    else:
        kernel = ConstantKernel() * Matern() + DotProduct()

    # Creating GP regressor
    model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=50, alpha=10, normalize_y=False)

    # Training the model
    model.fit(data_time.reshape(-1, 1), data)

    # Results
    target_pred = model.predict(data_time.reshape(-1, 1), return_std=False)

    original_data_reg[i] = target_pred * scale_factor
    
    print("\nLearned kernel: %s" % model.kernel_)

Comparing original and regularized data

In [ ]:
for k, (data, data_reg) in enumerate(zip(original_data, original_data_reg)):
    if k == 0:
        legend1 = "Dead individuals (cumulative)";
    elif k == 1:
        legend1 = "New confirmed cases";
    elif k == 2:
        legend1 = "Infected individuals (cumulative)";
    else:
        legend1 = "Recovered individuals (cumulative)";
        
    plotDatasetsComparison(
        data_time,
        data,
        data_reg,
        legend1,
        "Regularized data",
        "Time (days)",
        "Population",
        legend1
    )

Defining SEIRPDQ model

In [ ]:
@jit(nopython=True)
def seirpdq_model(
    t,
    X,
    beta,
    gamma_I,
    gamma_P,
    d_P,
    omega,
    rho,
    sigma,
    N=1,
):
    """
    SEIRPD-Q python implementation.
    """
    S, E, I, P, R, D, C, H = X
    S_prime = -beta / N * S * I - omega * S
    E_prime = beta / N * S * I - sigma * E - omega * E
    I_prime = sigma * rho * E - gamma_I * I - omega * I
    P_prime = sigma * (1 - rho) * E - d_P * P - gamma_P * P
    R_prime = gamma_I * I + gamma_P * P + omega * (S + E + I)
    D_prime = d_P * P
    C_prime = sigma * (1 - rho) * E
    H_prime = gamma_P * P
    return S_prime, E_prime, I_prime, P_prime, R_prime, D_prime, C_prime, H_prime

Defining the ODE solver

In [ ]:
def seirpdq_ode_solver(
    y0,
    t_span,
    t_eval,
    beta,
    omega,
    d_P,
    gamma_P=1 / 16.7,
    gamma_I=1 / 16.7,
    rho=0.6,
    sigma=1 / 5.8,
    N=1,
):
    solution_ODE = solve_ivp(
        fun=lambda t, y: seirpdq_model(
            t,
            y,
            beta=beta,
            gamma_I=gamma_I,
            gamma_P=gamma_P,
            d_P=d_P,
            omega=omega,
            rho=rho,
            sigma=sigma,
            N=N,
        ),
        t_span=t_span,
        y0=y0,
        t_eval=t_eval,
        method="LSODA",
    )

    return solution_ODE

Defining problem wrapper

In [ ]:
def seirpdq_least_squares_error_ode_y0(
    par, time_exp, f_exp, fitting_model, known_initial_conditions, total_population
):
    num_of_initial_conditions_to_fit = 1
    num_of_parameters = len(par) - num_of_initial_conditions_to_fit
    args, trial_initial_conditions = [
        par[:num_of_parameters],
        par[num_of_parameters:],
    ]
    E0 = trial_initial_conditions
    I0, P0, R0, D0, C0, H0 = known_initial_conditions
    S0 = total_population - (E0 + I0 + P0 + R0 + D0)
    initial_conditions = [S0, E0, I0, P0, R0, D0, C0, H0]

    f_exp1, f_exp2, f_exp3, f_exp4 = f_exp
    time_span = (time_exp.min(), time_exp.max())

    num_of_qoi = len(f_exp1)

    try:
        y_model = fitting_model(initial_conditions, time_span, time_exp, *args)
        simulated_time = y_model.t
        
        # dead_individuals, infected_individuals, confirmed_cases, recovered_cases
        simulated_ode_solution = y_model.y
        (
            _,
            _,
            simulated_qoi2,
            _,
            simulated_qoi4,
            simulated_qoi1,
            simulated_qoi3,
            _,
        ) = simulated_ode_solution

        residual1 = f_exp1 - simulated_qoi1  # Deaths (cumulative)
        residual2 = f_exp2 - simulated_qoi2  # Infected
        residual3 = f_exp3 - simulated_qoi3  # Cases (cumulative)
        residual4 = f_exp4 - simulated_qoi4  # Recovered (cumulative)

        # Watch out the weights
        first_term = 1.0 * np.sum(residual1 ** 2.0)
        second_term = 0.0 * np.sum(residual2 ** 2.0)
        third_term = 1.0 * np.sum(residual3 ** 2.0)
        fourth_term = 0.0 * np.sum(residual4 ** 2.0)
        
        objective_function = 1 / num_of_qoi * (first_term + second_term + third_term + fourth_term)
    except ValueError:
        objective_function = 1e15

    return objective_function

Defining the number of days for validation and calibration data type

In [ ]:
num_days_prediction = 7

data_type = "reg" # "original" or "reg"

Defining calibration/validation ratio

In [ ]:
min_calibration_data = 60
max_calibration_data = len(data_time) - num_days_prediction
step_calibration_data = 1

Calculating the number of problems to be solved

In [ ]:
n_runs = int(((max_calibration_data - min_calibration_data - 1) / step_calibration_data) + 1)

Casting data

In [ ]:
data_time = np.array(data_time) # Casting time data to numpy array

original_data = np.array(original_data) # Casting original data to numpy array

original_data_reg = np.array(original_data_reg) # Casting regularized data to numpy array
original_data_reg[np.where(original_data_reg < 0)] = 0 # Avoiding negative values

Setting known initial conditions

In [ ]:
I0, P0, R0, D0, C0, H0 = (
    int(float(confirmed_cases[0])),
    int(float(confirmed_cases[0])),
    int(float(recovered_cases[0])),
    int(float(dead_individuals[0])),
    int(float(confirmed_cases[0])),
    int(float(recovered_cases[0])),
)
y0_seirpdq = I0, P0, R0, D0, C0, H0

Model simulation

In [ ]:
def simulate_model(data_time, y0, pars):
    S0 = target_population - (pars[3] + y0[0] + y0[1] + y0[2] + y0[3])
    y0_sol = S0, pars[3], y0[0], y0[1], y0[2], y0[3], y0[4], y0[5]

    t0 = float(data_time.min())
    tf = float(data_time.max())

    solution_ODE = seirpdq_ode_solver(
        y0_sol, (t0, tf), data_time, pars[0], pars[1], pars[2]
    )

    t_solution_ODE, y_solution_ODE = (
        solution_ODE.t,
        solution_ODE.y,
    )
    
    return y_solution_ODE

Defining optimization algorithm search space

In [ ]:
bounds_seirpdq = [
    (0, 1e-6),   # beta
    (0, 0.025),  # omega
    (0, 0.025),  # d_P
    (0, 10000),  # E0
]

RMSE function definition

In [ ]:
def calcRMSE(data1, data2, num_days_prediction):
    diff_dead = data1[0, :] - data2[0, :]
    diff_infected = data1[1, :] - data2[1, :]
    diff_confirmed = data1[2, :] - data2[2, :]
    diff_recovered = data1[3, :] - data2[3, :]
    
    dead_residual = np.sqrt(np.power(np.sum(diff_dead), 2) / num_days_prediction)
    infected_residual = np.sqrt(np.power(np.sum(diff_infected), 2) / num_days_prediction) # Not used
    confirmed_residual = np.sqrt(np.power(np.sum(diff_confirmed), 2) / num_days_prediction)
    recovered_residual = np.sqrt(np.power(np.sum(diff_recovered), 2) / num_days_prediction) # Not used
    
    rmse = dead_residual + confirmed_residual
    return rmse

Initializing arrays to store results (original data)

In [ ]:
beta_values = np.zeros(n_runs)
omega_values = np.zeros(n_runs)
d_P_values = np.zeros(n_runs)
E0_values = np.zeros(n_runs)
RMSE_values = np.zeros(n_runs)

Solving the problem n_runs times

In [ ]:
x0 = np.array([3.2695e-8, 0.0137886, 0.0125293, 415.8004]) # This is know by prior executions (can also be random)
bounds = bounds_seirpdq

for p, num_data_calibration in enumerate(np.arange(min_calibration_data, max_calibration_data, step_calibration_data)):
    data_time_validation = data_time[num_data_calibration : (num_data_calibration + num_days_prediction)]
    validation_data = original_data[:, num_data_calibration : (num_data_calibration + num_days_prediction)]
    
    data_time_calibration = data_time[0 : num_data_calibration]
    if data_type == "original":
        calibration_data = original_data[:, 0 : num_data_calibration] # Using original data
    elif data_type == "reg":
        calibration_data = original_data_reg[:, 0 : num_data_calibration] # Using regularized dataset
    
    print("\nRun: %d/%d\n" %(p+1, n_runs))
    
    success_res = False
    
    while success_res == False:
        
        print(x0)
    
        result_NM = optimize.minimize(
            seirpdq_least_squares_error_ode_y0,
            x0=x0,
            args=(
                data_time_calibration,
                calibration_data,
                seirpdq_ode_solver,
                y0_seirpdq,
                target_population,
            ),
            method="Nelder-Mead",
            options={"maxiter": 500},
        )  
        
        if result_NM.success == True:
            sol_NM = result_NM.x
            for k in range(len(x0)):
                bounds[k] = (sol_NM[k] - 0.05 * sol_NM[k], sol_NM[k] + 0.05 * sol_NM[k])
        
            result_DE = optimize.differential_evolution(
                seirpdq_least_squares_error_ode_y0,
                bounds=bounds,
                args=(
                    data_time_calibration,
                    calibration_data,
                    seirpdq_ode_solver,
                    y0_seirpdq,
                    target_population,
                ),
                popsize=20,
                strategy="best1bin",
                tol=1e-6,
                recombination=0.95,
                mutation=0.6,
                maxiter=1000,
                polish=True,
                init="latinhypercube",
                disp=False,
                seed=seed,
                callback=None,
                updating="immediate",
                workers=1,
            )
            success_res = result_DE.success
        
        else:
            for i in range(len(bounds_seirpdq)):
                sign = np.random.uniform(0, 0.01, 1)
                shift = sign * x0[i]

                if np.random.uniform(0, 1, 1) < 0.5:
                    x0[i] = x0[i] - shift
                else:
                    x0[i] = x0[i] + shift

    print(result_DE)
    
    opt_pars = result_DE.x
    (
        beta_deterministic,
        omega_deterministic,
        d_P_deterministic,
        E0_deterministic
    ) = opt_pars

    beta_values[p] = beta_deterministic
    omega_values[p] = omega_deterministic
    d_P_values[p] = d_P_deterministic
    E0_values[p] = E0_deterministic

    y_solution_ODE = simulate_model(data_time, y0_seirpdq, opt_pars)
    
    (
        S_predict_seirpdq,
        E_predict_seirpdq,
        I_predict_seirpdq,
        P_predict_seirpdq,
        R_predict_seirpdq,
        D_predict_seirpdq,
        C_predict_seirpdq,
        H_predict_seirpdq,
    ) = y_solution_ODE

    validation_results = np.vstack((D_predict_seirpdq, I_predict_seirpdq, C_predict_seirpdq, R_predict_seirpdq))
    validation_results = validation_results[:, num_data_calibration : (num_data_calibration + num_days_prediction)]

    RMSE_values[p] = calcRMSE(validation_data, validation_results, num_days_prediction)
    
    x0 = opt_pars
    for i in range(len(bounds_seirpdq)):
        sign = np.random.uniform(0, 0.01, 1)
        shift = sign * x0[i]

        if np.random.uniform(0, 1, 1) < 0.5:
            x0[i] = x0[i] - shift
        else:
            x0[i] = x0[i] + shift
    

Saving optimal parameter values to CSV file

In [ ]:
pd.DataFrame(beta_values).to_csv(f"{OUTPUT_PATH}/beta_{data_type}.csv")
pd.DataFrame(omega_values).to_csv(f"{OUTPUT_PATH}/omega_{data_type}.csv")
pd.DataFrame(d_P_values).to_csv(f"{OUTPUT_PATH}/d_P_{data_type}.csv")
pd.DataFrame(E0_values).to_csv(f"{OUTPUT_PATH}/E0_{data_type}.csv")
pd.DataFrame(RMSE_values).to_csv(f"{OUTPUT_PATH}/RMSE_{data_type}.csv")

Plotting optimal parameter values

In [ ]:
array_n_runs = np.linspace(1, n_runs, num=n_runs)

plotDataset(array_n_runs, beta_values, "Run", "beta", "beta", data_type)
plotDataset(array_n_runs, omega_values, "Run", "omega", "omega", data_type)
plotDataset(array_n_runs, d_P_values, "Run", "d_P", "d_P", data_type)
plotDataset(array_n_runs, E0_values, "Run", "E0", "E0", data_type)

Comparing values of RMSE

In [ ]:
RMSE_values = pd.read_csv(f"{OUTPUT_PATH}/RMSE_original.csv", sep=',', usecols=[1]).to_numpy()
RMSE_values_reg = pd.read_csv(f"{OUTPUT_PATH}/RMSE_reg.csv", sep=',', usecols=[1]).to_numpy()

array_n_runs = np.linspace(1, n_runs, num=n_runs)

plotDatasetsComparison(
    array_n_runs,
    RMSE_values,
    RMSE_values_reg,
    "Original data",
    "Regularized data",
    "Run",
    "RMSE",
    "RMSE_original_reg"
)

Mean of RMSE

In [ ]:
print("RMSE (original data) = %f\n" %(np.mean(RMSE_values)))
print("RMSE (regularized data) = %f" %(np.mean(RMSE_values_reg)))

Variation of infected curves

In [ ]:
sols_confirmed = np.zeros((int(array_n_runs[-1]), int(data_time[-1] - min_confirmed + 1)))
sols_dead = np.zeros((int(array_n_runs[-1]), int(data_time[-1] - min_confirmed + 1)))

for p in np.array(array_n_runs, dtype=np.int):
    parameters = [beta_values[p - 1], omega_values[p - 1], d_P_values[p - 1]]
    
    S0 = target_population - (E0_values[p - 1] + I0 + P0 + R0 + D0)
    y0_sol = S0, E0_values[p - 1], I0, P0, R0, D0, C0, H0
    
    t0 = float(data_time.min())
    tf = float(data_time.max())
    
    solution_ODE = seirpdq_ode_solver(
        y0_sol, (t0, tf), data_time, *parameters
    )
    
    t_solution_ODE, y_solution_ODE = (
        solution_ODE.t,
        solution_ODE.y,
    )
    
    (
        _,
        _,
        _,
        _,
        _,
        D_predict_seirpdq,
        C_predict_seirpdq,
        _,
    ) = y_solution_ODE
    
    sols_confirmed[p - 1, :] = np.array((C_predict_seirpdq)).reshape((1, int(data_time[-1] - min_confirmed + 1)))
    sols_dead[p - 1, :] = np.array((D_predict_seirpdq)).reshape((1, int(data_time[-1] - min_confirmed + 1)))

pd.DataFrame(sols_confirmed).to_csv(f"{OUTPUT_PATH}/{data_type}_sols_confirmed.csv")
pd.DataFrame(sols_dead).to_csv(f"{OUTPUT_PATH}/{data_type}_sols_dead.csv")
    
if data_type == "original":
    confirmed_data = original_data[2, :]
    dead_data = original_data[0, :]
else:
    confirmed_data = original_data_reg[2, :]
    dead_data = original_data_reg[0, :]

plotDatasetsComparison(
    data_time,
    sols_confirmed.T,
    confirmed_data,
    None,
    "Original data of confirmed individuals",
    "Time (days)",
    "Population",
    "comparison_infected_cumulative_original")

plotDatasetsComparison(
    data_time,
    sols_dead.T,
    dead_data,
    None,
    "Original data of dead individuals",
    "Time (days)",
    "Population",
    "comparison_dead_cumulative_original")

Box plot of simulations

In [ ]:
original_sols_confirmed = pd.read_csv(f"{OUTPUT_PATH}/original_sols_confirmed.csv", sep=',', usecols=[1]).to_numpy()
reg_sols_confirmed = pd.read_csv(f"{OUTPUT_PATH}/reg_sols_confirmed.csv", sep=',', usecols=[1]).to_numpy()

fig, ax = plt.subplots(figsize=(7, 5))

labels = ['Original data', 'Regularized data']
data_sim_confirmed = np.array((np.vstack([original_sols_confirmed[:, -1], reg_sols_confirmed[:, -1]]))).T

bplot2 = ax.boxplot(data_sim_confirmed,
                     notch=True,  # notch shape
                     vert=True,  # vertical box alignment
                     patch_artist=True,  # fill with color
                     labels=labels,
                     whis=(0,100),)

plt.plot([0.5, 2.5], [original_data[2, -1], original_data[2, -1]])

plt.xlabel("Boxes")
plt.ylabel("Observed values")

plt.tight_layout()
plt.show()
# plt.savefig(f"{OUTPUT_PATH}/box_plot_confirmed_simulations_line.pdf")

Numerical results comparing variations on simulations using original and regularized data

In [ ]:
original_sols_dead = pd.read_csv(f"{OUTPUT_PATH}/original_sols_dead.csv", sep=',', usecols=[1]).to_numpy()
reg_sols_dead = pd.read_csv(f"{OUTPUT_PATH}/reg_sols_dead.csv", sep=',', usecols=[1]).to_numpy()

print("Min predicted confirmed (regularized data): %d" %int(np.min(reg_sols_confirmed[:, -1])))
print("Max predicted confirmed (regularized data): %d" %int(np.max(reg_sols_confirmed[:, -1])))

print("Min predicted dead (regularized data): %d" %int(np.min(reg_sols_dead[:, -1])))
print("Max predicted dead (regularized data): %d\n" %int(np.max(reg_sols_dead[:, -1])))

print("Min predicted confirmed (original data): %d" %int(np.min(original_sols_confirmed[:, -1])))
print("Max predicted confirmed (original data): %d" %int(np.max(original_sols_confirmed[:, -1])))

print("Min predicted dead (original data): %d" %int(np.min(original_sols_dead[:, -1])))
print("Max predicted dead (original data): %d" %int(np.max(original_sols_dead[:, -1])))

Setting up Bayesian calibration

In [ ]:
bayes_data_fit = "regularized_no_d_P"
num_data_calibration = 70

data_time_calibration = data_time[0 : num_data_calibration]
np_original_data = np.array(original_data)
if bayes_data_fit == "original":
    observations_to_fit = np.vstack([np_original_data[0, 0 : num_data_calibration], np_original_data[2, 0 : num_data_calibration]]).T
elif bayes_data_fit == "regularized_no_d_P":
    observations_to_fit = np.vstack([original_data_reg[0, 0 : num_data_calibration], original_data_reg[2, 0 : num_data_calibration]]).T

ODE wrapper for bayesian calibration

In [ ]:
@theano.compile.ops.as_op(
    itypes=[
        t.dvector,
        t.dvector,
        t.dscalar,
        t.dscalar,  # beta
        t.dscalar,  # omega
        t.dscalar,  # d_P
        t.dscalar,  # E0
    ],
    otypes=[t.dmatrix],
)
def seirpdq_ode_wrapper_with_y0(
    time_exp, initial_conditions, total_population, beta, omega, d_P, E0
):
    time_span = (time_exp.min(), time_exp.max())
    args = [beta, omega, d_P]

    S0 = total_population - (
        E0 + initial_conditions[0] + initial_conditions[1] + initial_conditions[2] + initial_conditions[3]
    )
    ICs = (
        S0,
        E0,
        initial_conditions[0],  # I
        initial_conditions[1],  # P
        initial_conditions[2],  # R
        initial_conditions[3],  # D
        initial_conditions[4],  # C
        initial_conditions[5],  # H
    )

    y_model = seirpdq_ode_solver(ICs, time_span, time_exp, *args)
    simulated_time = y_model.t
    simulated_ode_solution = y_model.y
    (_, _, _, _, _, simulated_qoi1, simulated_qoi2, _,) = simulated_ode_solution

    concatenate_simulated_qoi = np.vstack([simulated_qoi1, simulated_qoi2]).T

    return concatenate_simulated_qoi

Performing Bayesian calibration

In [ ]:
draws = 3000
start_time = time.time()
percent_calibration = 0.95

y0_seirpdq = np.array([I0, P0, R0, D0, C0, H0], dtype=np.float64)

with pm.Model() as model_mcmc:

    beta = pm.Uniform(
        "beta", 
        lower=0, 
        upper=1e-6,
    )
    omega = pm.Uniform(
        "omega", 
        lower=0, 
        upper=0.02,
    )
    d_P = pm.Uniform(
        "d_P",
        lower=0,
        upper=0.02,
    )
    E0_uncertain = pm.Uniform(
        "E0",
        lower=0,
        upper=10000
    )

    standard_deviation = pm.Uniform(
        "std_deviation",
        lower=1e0,
        upper=1e4,
        shape=2
    )

    fitting_model = pm.Deterministic(
        "seirpdq_model",
        seirpdq_ode_wrapper_with_y0(
            theano.shared(data_time_calibration),
            theano.shared(np.array(y0_seirpdq)),
            theano.shared(target_population),
            beta,
            omega,
            d_P,
            E0_uncertain
        ),
    )

    likelihood_model = pm.Normal(
        "likelihood_model",
        mu=fitting_model,
        sigma=standard_deviation,
        observed=observations_to_fit
    )

    seirdpq_trace_calibration = pm.sample_smc(
        draws=draws,
        n_steps=25,
        parallel=True,
        cores=8,
        progressbar=True,
        random_seed=seed
    )

Functions for posterior calculation

In [ ]:
def calculate_rv_posterior_mpv(pm_trace, variable_names: list) -> dict:
    rv_mpv_values_dict = dict()
    progress_bar = tqdm(variable_names)
    for variable in progress_bar:
        progress_bar.set_description(f"Calulating MPV from KDE for {variable}")
        rv_realization_values = pm_trace[f"{variable}"]

        try:
            num_of_dimensions = rv_realization_values.shape[1]
        except IndexError:
            num_of_dimensions = 0

        if num_of_dimensions == 0:
            rv_mpv_value = _scalar_rv_mvp_estimation(rv_realization_values)
            rv_mpv_values_dict[f"{variable}"] = rv_mpv_value
        else:
            for dimension in range(num_of_dimensions):
                variable_name_decomposed = f"{variable}[{dimension}]"
                rv_realization_values_decomposed = np.array(rv_realization_values[:, dimension])
                rv_mpv_value = _scalar_rv_mvp_estimation(rv_realization_values_decomposed)
                rv_mpv_values_dict[f"{variable_name_decomposed}"] = rv_mpv_value

    return rv_mpv_values_dict


def _scalar_rv_mvp_estimation(rv_realization_values: np.ndarray) -> np.ndarray:
    num_of_realizations = len(rv_realization_values)
    kernel = gaussian_kde(rv_realization_values)
    equally_spaced_samples = np.linspace(
        rv_realization_values.min(),
        rv_realization_values.max(),
        num_of_realizations
    )
    kde = kernel(equally_spaced_samples)
    kde_max_index = np.argmax(kde)
    rv_mpv_value = equally_spaced_samples[kde_max_index]
    return rv_mpv_value


def add_mpv_to_summary(arviz_summary: pd.DataFrame, rv_modes_dict: dict) -> pd.DataFrame:
    new_arviz_summary = arviz_summary.copy()
    variable_names = list(rv_modes_dict.keys())
    rv_mode_values = list(rv_modes_dict.values())
    new_arviz_summary["mpv"] = pd.Series(data=rv_mode_values, index=variable_names)
    return new_arviz_summary

Posterior plotting

In [ ]:
calibration_variable_names = [
    "std_deviation",
    "beta",
    "omega",
    "d_P",
    "E0"
]

plot_step = 1

progress_bar = tqdm(calibration_variable_names)
for variable in progress_bar:
    progress_bar.set_description("Arviz post-processing")
    pm.traceplot(seirdpq_trace_calibration[::plot_step], var_names=(f"{variable}"))
    plt.savefig(f"{OUTPUT_PATH}/seirpdq_{variable}_traceplot_cal_{bayes_data_fit}_{num_data_calibration}.pdf")

    pm.plot_posterior(
        seirdpq_trace_calibration[::plot_step], 
        var_names=(f"{variable}"), 
        kind="hist", 
        round_to=5,
        point_estimate="mode"
    )
    plt.savefig(f"{OUTPUT_PATH}/seirpdq_{variable}_posterior_cal_{bayes_data_fit}_{num_data_calibration}.pdf")

Post-processing stats

In [ ]:
df_stats_summary = az.summary(
    data=seirdpq_trace_calibration,
    var_names=calibration_variable_names,
    kind='stats',
    round_to=15,
)
calibration_variable_modes = calculate_rv_posterior_mpv(
    pm_trace=seirdpq_trace_calibration, variable_names=calibration_variable_names
)
df_stats_summary = add_mpv_to_summary(df_stats_summary, calibration_variable_modes)
df_stats_summary.to_csv(f"{OUTPUT_PATH}/stats_summary_calibration_{bayes_data_fit}_{num_data_calibration}.csv")

Marginal distributions

In [ ]:
az.plot_pair(
    seirdpq_trace_calibration,
    var_names=calibration_variable_names[1:],
    kind="hexbin",
    fill_last=False,
    figsize=(10, 8),
)
plt.savefig(f"{OUTPUT_PATH}/seirpdq_marginals_cal_{bayes_data_fit}_{num_data_calibration}.pdf")

Plotting Bayesian calibration

In [ ]:
percentile_cut = 2.5

y_min = np.percentile(seirdpq_trace_calibration["seirpdq_model"], percentile_cut, axis=0)
y_max = np.percentile(seirdpq_trace_calibration["seirpdq_model"], 100 - percentile_cut, axis=0)
y_fit = np.percentile(seirdpq_trace_calibration["seirpdq_model"], 50, axis=0)

std_deviation = seirdpq_trace_calibration.get_values("std_deviation")
sd_pop = np.sqrt(std_deviation.mean())

data_time_calibration = data_time[0 : num_data_calibration]
calibration_data = (observations_to_fit).T

data_time_validation = data_time[num_data_calibration : (num_data_calibration + num_days_prediction)]
validation_data = np_original_data[:, num_data_calibration : (num_data_calibration + num_days_prediction)]

# %%
plt.figure(figsize=(9, 7))

plt.plot(
    data_time_calibration,
    y_fit[:, 0],
    "r",
    label="Deaths (SEIRPD-Q)",
    marker="D",
    linestyle="-",
    markersize=2,
)
plt.fill_between(data_time_calibration, y_min[:, 0], y_max[:, 0], color="r", alpha=0.2)

plt.plot(
    data_time_calibration,
    y_fit[:, 1],
    "b",
    label="Cases (SEIRPD-Q)",
    marker="v",
    linestyle="-",
    markersize=2,
)
plt.fill_between(data_time_calibration, y_min[:, 1], y_max[:, 1], color="b", alpha=0.2)

plt.plot(
    data_time_calibration, calibration_data[1, :], label="Confirmed data", marker="s", linestyle="", markersize=2
)
plt.plot(
    data_time_calibration, calibration_data[0, :], label="Recorded deaths", marker="v", linestyle="", markersize=2
)

plt.xlabel("Time (days)")
plt.ylabel("Population")
plt.legend()
plt.grid()

plt.tight_layout()

# plt.savefig(f"{OUTPUT_PATH}/seirpdq_calibration_bayes_{bayes_data_fit}_{num_data_calibration}.pdf")

Bayesian prediction

In [ ]:
dict_realizations = dict()
progress_bar = tqdm(calibration_variable_names[1:])
for variable in progress_bar:
    progress_bar.set_description(f"Gathering {variable} realizations")
    parameter_realization = seirdpq_trace_calibration.get_values(f"{variable}")
    dict_realizations[f"{variable}"] = parameter_realization

df_realizations = pd.DataFrame(dict_realizations)
df_realizations.to_csv(f"{OUTPUT_PATH}/calibration_realizations_{bayes_data_fit}_{num_data_calibration}.csv")

S_predicted = list()
E_predicted = list()
I_predicted = list()
P_predicted = list()
R_predicted = list()
D_predicted = list()
C_predicted = list()
H_predicted = list()

data_time_prediction = np.hstack([data_time_calibration, data_time_validation])
t0 = np.min(data_time_prediction)
tf = np.max(data_time_prediction)

number_of_total_realizations = len(dict_realizations["beta"])
for realization in trange(number_of_total_realizations):
    parameters_realization = [
        realization_reg[realization, 0],
        realization_reg[realization, 1],
        realization_reg[realization, 2],
    ]
    y0_estimated = (
        target_population - (realization_reg[realization, 3] + I0 + P0 + R0 + D0),
        realization_reg[realization, 3],
        I0,
        P0,
        R0,
        D0,
        C0,
        H0,
    )
    solution_ODE_predict = seirpdq_ode_solver(
        y0_estimated, (t0, tf), data_time_prediction, *parameters_realization
    )
    t_computed_predict, y_computed_predict = solution_ODE_predict.t, solution_ODE_predict.y
    S, E, I, P, R, D, C, H = y_computed_predict

    S_predicted.append(S)
    E_predicted.append(E)
    I_predicted.append(I)
    P_predicted.append(P)
    R_predicted.append(R)
    D_predicted.append(D)
    C_predicted.append(C)
    H_predicted.append(H)

S_predicted = np.array(S_predicted)
E_predicted = np.array(E_predicted)
I_predicted = np.array(I_predicted)
P_predicted = np.array(P_predicted)
R_predicted = np.array(R_predicted)
D_predicted = np.array(D_predicted)
C_predicted = np.array(C_predicted)
H_predicted = np.array(H_predicted)

percentile_cut = 2.5
S_min = np.percentile(S_predicted, percentile_cut, axis=0)
S_max = np.percentile(S_predicted, 100 - percentile_cut, axis=0)
S_mean = np.percentile(S_predicted, 50, axis=0)

E_min = np.percentile(E_predicted, percentile_cut, axis=0)
E_max = np.percentile(E_predicted, 100 - percentile_cut, axis=0)
E_mean = np.percentile(E_predicted, 50, axis=0)

I_min = np.percentile(I_predicted, percentile_cut, axis=0)
I_max = np.percentile(I_predicted, 100 - percentile_cut, axis=0)
I_mean = np.percentile(I_predicted, 50, axis=0)

P_min = np.percentile(P_predicted, percentile_cut, axis=0)
P_max = np.percentile(P_predicted, 100 - percentile_cut, axis=0)
P_mean = np.percentile(P_predicted, 50, axis=0)

R_min = np.percentile(R_predicted, percentile_cut, axis=0)
R_max = np.percentile(R_predicted, 100 - percentile_cut, axis=0)
R_mean = np.percentile(R_predicted, 50, axis=0)

D_min = np.percentile(D_predicted, percentile_cut, axis=0)
D_max = np.percentile(D_predicted, 100 - percentile_cut, axis=0)
D_mean = np.percentile(D_predicted, 50, axis=0)

C_min = np.percentile(C_predicted, percentile_cut, axis=0)
C_max = np.percentile(C_predicted, 100 - percentile_cut, axis=0)
C_mean = np.percentile(C_predicted, 50, axis=0)

Plotting Bayesian prediction

In [ ]:
fig, ax1 = plt.subplots(figsize=(9, 7))

ax1.plot(
    t_computed_predict,
    C_mean,
    "r",
    label="Confirmed cases",
    marker="o",
    linestyle="-",
    markersize=0,
    linewidth=0.5,
)
ax1.fill_between(t_computed_predict, C_min, C_max, color="r", alpha=0.2)

ax1.plot(
    data_time_calibration, calibration_data[1, :], label="Confirmed data (calibration)", marker="o", linestyle="", markersize=3, color='C1'
)

ax1.plot(
    data_time_validation, validation_data[2, :], label="Confirmed data (validation)", marker="o", linestyle="", markersize=6
)

ax2 = ax1.twinx() 

ax2.plot(
    t_computed_predict,
    I_mean,
    "g",
    label="Infected",
    marker="o",
    linestyle="-",
    markersize=0,
    linewidth=0.5,
)
ax2.fill_between(t_computed_predict, I_min, I_max, color="c", alpha=0.2)

ax2.plot(
    t_computed_predict,
    P_mean,
    "b",
    label="Positively diagnosed",
    marker="o",
    linestyle="-",
    markersize=0,
    linewidth=0.5,
)
ax2.fill_between(t_computed_predict, P_min, P_max, color="b", alpha=0.2)

ax1.plot(
    t_computed_predict,
    D_mean,
    "k",
    label="Dead individuals",
    marker="o",
    linestyle="-",
    markersize=0,
    linewidth=0.5,
)
ax1.fill_between(t_computed_predict, D_min, D_max, color="k", alpha=0.2)

ax1.plot(
    data_time_calibration, calibration_data[0, :], label="Recorded deaths (calibration)", marker="o", linestyle="", markersize=3
)

ax2.plot(
    data_time_validation, validation_data[0, :], label="Recorded deaths (validation)", marker="o", linestyle="", markersize=6
)

ax1.set_xlabel("Time (days)")
ax1.set_ylabel("Population")

ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
ax1.grid()

fig.tight_layout()

# plt.savefig(f"{OUTPUT_PATH}/seirpdq_prediction_cumulative_{bayes_data_fit}_{num_data_calibration}_scaled.pdf")